In [16]:
# Turn them into embeddings (vectors).


from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import os
import numpy as np

# Load CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", use_safetensors = True)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
    embedding = embedding.cpu().numpy().flatten()
    return embedding / np.linalg.norm(embedding)  # normalize


In [17]:
# Uses cosine similarity to recommend the most relevant next image.



import numpy as np

# ---------------------------
# Helper functions
# ---------------------------

def unit(x):
    """Normalize a vector to unit length."""
    norm = np.linalg.norm(x)
    if norm == 0:
        return x
    return x / norm

def dot(a, b):
    """Dot product (cosine similarity if both are unit vectors)."""
    return np.dot(a, b)


# ---------------------------
# Recommendation System
# ---------------------------

class ImageRecommender:
    def __init__(self, image_embeddings, alpha=0.9, eta=1.0):
        """
        image_embeddings : list or np.array of shape (N, D)
            Precomputed embeddings for N images, each of dimension D.
        alpha : float
            Weight for old preference vector.
        eta : float
            Learning rate (weight for new feedback).
        """
        self.image_embeddings = np.array(image_embeddings)
        self.N, self.D = self.image_embeddings.shape
        self.alpha = alpha
        self.eta = eta
        self.preference = np.zeros(self.D)  # start with zero preference

        # Keep track of seen images
        self.seen = set()

    def update_preference(self, chosen_idx, feedback=1):
        """Update preference vector given user feedback."""
        e_chosen = self.image_embeddings[chosen_idx]
        self.preference = unit(self.alpha * self.preference + self.eta * feedback * e_chosen)
        self.seen.add(chosen_idx)

    def recommend_next(self):
        """Recommend the next image based on current preference vector."""
        scores = []
        for idx, e in enumerate(self.image_embeddings):
            if idx in self.seen:
                continue
            score = dot(self.preference, e)
            scores.append((idx, score))

        # Pick image with highest score
        if scores:
            next_idx, best_score = max(scores, key=lambda x: x[1])
            return next_idx, best_score
        else:
            return None, None


In [ ]:
chosen_idx = 0  # example
rec.update_preference(chosen_idx=chosen_idx, feedback=1)

# Show the current liked image
print("User liked:", paths[chosen_idx])

from PIL import Image


User liked: /Users/abdul/Downloads/Project/photos/3b6f33f2.jpg


In [ ]:
next_idx, score = rec.recommend_next()
print("Next recommended image:", paths[next_idx], "with score:", score)


Next recommended image: /Users/abdul/Downloads/Project/photos/018b51ef.jpg with score: 0.8870359409212728
